# How to sanction a character

With automated steps, where possible.

If running the automated steps, run each bit of python code in order. Check the description, since some steps are not possible to automate (like copying the original sheet, or creating a discord channel).

## Python setup

### Imports

In [37]:
from __future__ import print_function

import os.path
import re

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from urllib.parse import urlparse, parse_qs, quote

from datetime import datetime

# If modifying these scopes, delete the file token.json.
SCOPES = [
    #'https://www.googleapis.com/auth/documents.readonly',
    #'https://www.googleapis.com/auth/spreadsheets.readonly',
    'https://www.googleapis.com/auth/drive']

### Functions

In [39]:
def get_credentials():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return creds

def get_service_object(service):
    creds = get_credentials()
    try:
        return get_service(service, creds)
    except HttpError as err:
        print(err)
        
def get_service(service, creds):
    """ Match requires python 3.10 """
    #match service:
    #    case 'sheets':
    #        return build('sheets', 'v4', credentials=creds)
    #    case 'docs':
    #        return build('docs', 'v1', credentials=creds)
    #    case _:
    #        raise Exception(f'Service: {service} not implemented')
    if (service.casefold() == 'sheets'):
        return build('sheets', 'v4', credentials=creds)
    elif (service.casefold() == 'docs'):
        return build('docs', 'v1', credentials=creds)
    elif (service.casefold() == 'drive'):
        return build('drive', 'v3', credentials=creds)
        
    raise Exception(f'Service: {service} not implemented')
    
def build_url(type, id):
    return f'https://docs.google.com/{type}/d/{id}'

def get_template_ids(game, sheet_id):
    cell_range = 'Overview!W5:W6'

    template_cells = get_service_object('sheets').spreadsheets().values().get(spreadsheetId=sheet_id, range=cell_range).execute()
    values = template_cells.get('values', [])
    
    return {'st': get_id_from_drive_url(values[0][0]), 'player': get_id_from_drive_url(values[1][0])}

def get_id_from_drive_url(drive_url):
    parsed_url = urlparse(drive_url)
    qs = parse_qs(parsed_url.query)
    
    if id in qs:
        return qs['id'][0]
    bits = parsed_url.path.split('/')
    
    return bits[-1]

def character_name_from_document(document):
    title = document.get('title')
    title_chunks = title.split('/')
    
    return title_chunks[0].strip()

def set_editor_permissions(e_mail, file_id):
    new_permissions = {
        'type': 'group',
        'role': 'writer',
        'emailAddress': e_mail
    }

    permission_response = get_service_object('drive').permissions().create(
          fileId=file_id, body=new_permissions).execute()

def set_viewer_permissions(e_mail, file_id):
    new_permissions = {
        'type': 'group',
        'role': 'viewer',
        'emailAddress': e_mail
    }

    permission_response = get_service_object('drive').permissions().create(
          fileId=file_id, body=new_permissions).execute()

def add_character_row(game, spreadsheet_id, st_sheet_id):
    if game not in game_dict.keys():
        return None

    if game == 'scion':
        list_range = 'Character list!A2:O'
        
        row = [
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B1")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E2")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B2")',
            build_url('spreadsheets', st_sheet_id),
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B5")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!C1")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B7")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!Q2")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!Q3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!Q4")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E9")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!I9")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Deeds!U1")'
        ]
        
    elif game == 'gunstar':
        list_range = 'Character list!A2:I'
        row = [
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B1")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E2")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B2")',
            build_url('spreadsheets', st_sheet_id),
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B5")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!C1")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B7")'
        ]
        
    elif game == 'exalted':
        list_range = 'Character list!A2:I'
        row = [
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B1")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E2")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!E3")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B2")',
            build_url('spreadsheets', st_sheet_id),
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B5")',
            '=IMPORTRANGE(INDIRECT(CONCAT("F", ROW())), "Overview!B7")',
            'Discord'
        ]
    
    # Values will be appended after the last row of the table.
    value_range_body = {
        'values': [ row ]
    }

    return sheet_service.spreadsheets().values().append(
        spreadsheetId = spreadsheet_id,
        range = list_range,
        valueInputOption = 'USER_ENTERED',
        body = value_range_body
    ).execute()

def get_floor_xp(game, sheet_id):
    if game == 'exalted':
        return None

    floor_xp = get_service_object('sheets').spreadsheets().values().get(
        spreadsheetId = sheet_id, range = "'XP log'!J2"
    ).execute()

    xp = floor_xp.get('values', [])[0][0]
    
    if game == 'scion':
        return { 
           'cell': "'XP log'!J2",
           'xp': xp
       }
    elif game == 'gunstar':
        return { 
           'cell': "'XP log'!L2",
           'xp': xp
       }
    
    return None

def get_wiki_url(game, character_name):
    if game == 'scion':
        prefix = 'ScD'
    elif game == 'gunstar':
        prefix = 'ExG'
    elif game == 'exalted':
        prefix = 'ExND'
        
    return f'https://wiki.mythic-saga.com/view/{prefix}:{quote(character_name)}'


## Step 1
With the character sheet to be sanctioned, copy it to the right folder and rename if necessary, ie:

1. `File => Make a copy`
2. Folder `LegendaryDeeds/<game folder>/Character sheets` with the name `<Character name> / <appropriate info>`, ex "Islay Duncans / Aesir of Ran" or "Sirius / Godblooded"
3. Make sure to set Share permissions so that anyone with the link can view
4. Copy the file ID into the variable of next step
5. Uncomment the right game

### Variables

#### Editable variables

In [40]:
character_sheet = '1vdwITGp672ojI2YtgkC-pu0CVaGxJf8EO5lq83aFisg'

# Set the player's google e-mail here
player_email = 'Majorobservation@gmail.com'

# Games
#game = 'scion'
game = 'gunstar'
#game = 'exalted'

# Who sanctioned the character? a string
sanctioning_st = 'Marie'

# If the game is Scion, the callings is a list of the character's callings
callings = ['<calling1>', '<calling2>', '<calling3>']

#### Script variables

In [41]:
game_dict = {
    'scion': '1y9sphl371sxBA4Zz-3JrZAzN6x0XqzhL80ptfnLNGsw',
    'gunstar': '10DzZArk5SYyIiYgwuX1qz16wfO-JZt4SpNVY7JETp7g',
    'exalted': '16WOXfs4KtXL5qGHDRJDR07uhxexJzHC7-i22wB7V0Nk'
}

sheet_service = get_service_object('sheets')
document_service = get_service_object('docs')
drive_service = get_service_object('drive')

masterlist_url = build_url('spreadsheets', game_dict[game])
character_url = build_url('document', character_sheet)

name = character_name_from_document(document_service.documents().get(documentId=character_sheet).execute())

print(f'New character is named: {name}')
print(f'Character masterlist: { masterlist_url }')
print(f'Character sheet: {character_url}')

# note: If you get invalid_grant, remove the token.json to reauthorize!

New character is named: Helcami of Ylaset
Character masterlist: https://docs.google.com/spreadsheets/d/10DzZArk5SYyIiYgwuX1qz16wfO-JZt4SpNVY7JETp7g
Character sheet: https://docs.google.com/document/d/1vdwITGp672ojI2YtgkC-pu0CVaGxJf8EO5lq83aFisg


## Step 2

Next set up the various files needed. If manually, the steps are as follows:

1. Open the character masterlist (link above)
2. On the Overview tab, open the ST sheet and Player sheet
3. Copy both (see step 1), with the title in each having the character's name where the template is named Template, ex `Sirius ST sheet Gunstar``
4. Get the e-mail from the player and add the player as editor of the Player Sheet
5. Copy a row on the masterlist, replacing the ST sheet id with that of the ST sheet you just copied


In [42]:
# Get id to ST and player sheets
template_ids = get_template_ids(game, game_dict[game])
sheet_folder_response = drive_service.files().get(fileId=template_ids['st'], fields='parents').execute()
sheet_folder_id = sheet_folder_response['parents'][0]
st_template = drive_service.files().get(fileId=template_ids['st']).execute()
player_template = drive_service.files().get(fileId=template_ids['player']).execute()

st_file_name = st_template['name'].replace('<Template>', name)
player_file_name = player_template['name'].replace('<Template>', name)

# 2.3
player_sheet_id = None
st_sheet_id = None

if st_sheet_id is None:
    print("Creating new ST sheet")
    st_copy = drive_service.files().copy(fileId=template_ids['st'], body={'parents': [sheet_folder_id], 'name': st_file_name} ).execute()
    st_sheet_id = st_copy['id']

if player_sheet_id is None:
    print("Creating new player sheet")
    player_copy = drive_service.files().copy(fileId=template_ids['player'], body={'parents': [sheet_folder_id], 'name': player_file_name} ).execute()
    player_sheet_id = player_copy['id']

player_sheet_url = build_url('spreadsheets', player_sheet_id)
st_sheet_url = build_url('spreadsheets', st_sheet_id)
print(f"{player_sheet_url=}, {st_sheet_url=}")

# 2.4
set_editor_permissions(player_email, player_sheet_id)

# 2.5
add_character_row(game, game_dict[game], st_sheet_id)

Creating new ST sheet
Creating new player sheet
player_sheet_url='https://docs.google.com/spreadsheets/d/1T5ohs7dc6dYuRcirwoXTBiefm50Y0z93UK75btGiv5Q', st_sheet_url='https://docs.google.com/spreadsheets/d/1WI0sg1TzqVpPYbv_Zsf6WMLB_oyjn7WowmJfEm2MDrM'


{'spreadsheetId': '10DzZArk5SYyIiYgwuX1qz16wfO-JZt4SpNVY7JETp7g',
 'tableRange': "'Character list'!A1:I17",
 'updates': {'spreadsheetId': '10DzZArk5SYyIiYgwuX1qz16wfO-JZt4SpNVY7JETp7g',
  'updatedRange': "'Character list'!A18:I18",
  'updatedRows': 1,
  'updatedColumns': 9,
  'updatedCells': 9}}

## Step 3

If you're doing this manually:

**On the ST sheet**, replace the link to "ST Sheet" (cell B6) with the link to itself, the link to the character sheet (B2) with the doc link of the sheet, and the player sheet link (B5) with the player sheet link. Also fill in the character's name in cell B1 and their sanctioning date in cell B3.

**On the player sheet**, replace the link in the `IMPORTRANGE` with one to the ST sheet.

For XP, make sure that the `XP Log` tab includes any bonus starting XP.

In [43]:
do_ST_sheet = True
do_player_sheet = True

if do_ST_sheet:
    # ST sheet, XP log
    now = datetime.now()

    values = [
        [name],
        [character_url],
        [now.strftime('%m/%d/%Y')],
        [sanctioning_st],
        [player_sheet_url],
        [st_sheet_url]
    ]
    character_data = [
        {'range': 'Overview!B1:B6', 'values': values}
    ]

    floor_xp = get_floor_xp(game, game_dict[game])

    if floor_xp is not None:
        character_data.append(
            {'range': floor_xp['cell'], 'values': [[floor_xp['xp']]]}
        )

    if game == 'scion':
        calling_values = [
            [callings[0]],
            [callings[1]],
            [callings[2]]
        ]
        character_data.append(
            {'range': 'Overview!Q2:Q4', 'values': calling_values}
        )

    batch_body = {
        'valueInputOption': 'USER_ENTERED',
        'data': character_data
    }

    sheet_service.spreadsheets().values().batchUpdate(spreadsheetId=st_sheet_id, body=batch_body).execute()

if do_player_sheet:
    # Player sheet
    values = [
      [f'=IMPORTRANGE("{st_sheet_url}", "Overview!A1:P20")']
    ]

    data = { 'values' : values }

    sheet_service.spreadsheets().values().update(
        spreadsheetId = player_sheet_id, 
        body = data, 
        range = 'Overview--read only!A1', 
        valueInputOption='USER_ENTERED').execute()

## Step 4
Wiki! The link is in cell C1(created from the name), so you need to create the wiki and fill in base information (in particular name and such)

In [44]:
wiki_link = get_wiki_url(game, name)

print(f'Wiki link: {wiki_link}')

Wiki link: https://wiki.mythic-saga.com/view/ExG:Helcami%20of%20Ylaset


## Step 5

Create a Discord channel under the game's `Characters` category, with the name of the character. Explicitly add the player to the room, and pin a message with the following content (replace as necessary):


> ** QUICK LINKS **
>
> Character sheet: (link)
>
> Request sheet: (link)
>
> ST spreadsheet (view-only): (link)
>
> Wiki: (link)
>
> Connecting e-mail: (e-mail)

In [45]:
print('** QUICK LINKS **')
print(f'Character sheet: {character_url}')
print()
print(f'Request sheet: {player_sheet_url}')
print()
print(f'ST spreadsheet (view-only): {st_sheet_url}')
print()
print(f'Wiki: {wiki_link}')
print()
print(f'Connecting e-mail: {player_email}')

** QUICK LINKS **
Character sheet: https://docs.google.com/document/d/1vdwITGp672ojI2YtgkC-pu0CVaGxJf8EO5lq83aFisg

Request sheet: https://docs.google.com/spreadsheets/d/1T5ohs7dc6dYuRcirwoXTBiefm50Y0z93UK75btGiv5Q

ST spreadsheet (view-only): https://docs.google.com/spreadsheets/d/1WI0sg1TzqVpPYbv_Zsf6WMLB_oyjn7WowmJfEm2MDrM

Wiki: https://wiki.mythic-saga.com/view/ExG:Helcami%20of%20Ylaset

Connecting e-mail: Majorobservation@gmail.com
